# *Images*
## Paquete para imágenes

## *Images*

El paquete **Images** permite crear y manipular imágenes.

### Instalación y llamada

En **JuliaBox**, **Images** ya está instalado.

In [ ]:
#Pkg.update()
#Pkg.add("ImageMagick")
#Pkg.add("Images")

In [ ]:
using Images, Colors

## Creando imágenes

Las imágenes son arreglos $2$-dimensionales de "colores".

Crear una imágen es igual que crear una matriz, pero con entradas de color...

### Imágenes en grises

Para imágenes en grises, $0$ es el color negro y $1$ el blanco. Números entre $0$ y $1$ son los distintos tonos de grises..

In [ ]:
# Crea una imagen en grises (aleatoria)
# de 400x200 pixeles (400 a lo ancho y 200 a lo alto)
img_rand_gray = Gray.(rand(200,400))

Una vez creadas, las imágenes se pueden modificar...

In [ ]:
# Crea una imagen con puros 1, es decir, de color blanco
img = Gray.(ones(100,100)) # ones(Gray{Float16},100,100)

# En las entradas (i,i), pone 0, es decir, un pixel negro
for i in 1:100
    img[i,i] = 0.
end

img # Nótese que las imágenes se recorren de izquierda a derecha, pero de arriba a abajo

### Ejemplo: El diagrama de órbitas de una familia de funciones

El *diagrama de órbitas* de una familia de funciones $f_\lambda:[a,b]\rightarrow [a,b]$ consiste en dibujar los últimos $N$ puntos del conjunto de iteraciones $\{x_0,f_\lambda(x_0),\dots,f^M_\lambda(x_0),\dots,f^{M+N}_\lambda(x_0)\}$ sobre una línea vertical correspondiente a un $\lambda \in [\lambda_0,\lambda_1]$. Si la función $f_\lambda$ tiene órbitas atractoras, los puntos dibujados serán muy cercanos a ellas.

In [ ]:
function createOrbitDiagram(f::Function, x_min::Real, x_max::Real, lambda_min::Real, lambda_max::Real,
    width::Integer=500, height::Integer=500, n_approx::Integer=200, n_points::Integer=200)

    img_orbits = Gray.(ones(height, width)) # Imagen blanca

    for w_pixel in 1:width # Ciclo sobre los pixeles a lo ancho
        
        λ = w_pixel*(lambda_max-lambda_min)/width + lambda_min
    
        x = (x_min+x_max)/2 # Punto en el intervalo [x_min,x_max]
        
        for n in 1:n_approx # Calcula los primeros "n_approx" elementos de la orbita, para aproximarse a los atractores
            x = f(x,λ)
        end
  
        if x > 100. || x == Inf || x == -Inf
            continue # Si x es muy grande o "infinito", usa un nuevo lambda...
        end        
        
        h_pixel = 0 # Pixel a lo alto
    
        for n in 1:n_points # Dibuja los últimos "n_points" de la órbita de x 
            if x > 100. || x == Inf || x == -Inf
                break # Si x es muy grande o "infinito", termina este ciclo
            end        
            
            h_pixel = height - floor(Int64, height*(x-x_min)/(x_max-x_min)) # Las imágenes se recorrren de arriba a abajo...
        
            img_orbits[h_pixel, w_pixel] = 0. # Pone negro en el pixel
        
            x = f(x,λ)
        end
    end

    img_orbits # Regresa la imagen con las órbitas atractoras en negro
end

In [ ]:
createOrbitDiagram((x,λ) -> λ*x*(1.-x), 0.,1., 1,4., 2000, 2000, 400, 800)
#3.83,3.86 # 2000, 2000, 400, 800
#3.83,4., 2000, 2000, 400, 800

In [ ]:
createOrbitDiagram((x,λ) -> x<0.5 ? λ*x : λ*(1.-x), 0., 1., 1., 2.)

In [ ]:
createOrbitDiagram((x,λ) -> mod2pi(x+λ*sin(π*x)), 0,2π, -2π,2π)

### Ejemplo: Sistemas de funciones iteradas

Tómese un conjunto finito de transformaciones afines contractoras $f_n:\mathbb{R}^2\rightarrow \mathbb{R}^2$, es decir, cada función es de la forma $f_n(x,y)=(ax+by+e,cx+dy+f)$, y cumple que $||f_n(p)-f_n(q)||\leq ||p-q||$.

Existe un conjunto $A\subset \mathbb{R}^2$ (llamado el atractor) que es invariante para cualquier composición de las transformaciones.

In [ ]:
function createIFSAttractor(coeffs::Array, width::Integer=500, height::Integer=500,
        n_points::Integer=100000)

    n_fns, n_cols = size(coeffs) # coeffs debe ser una matriz de n_fns renglones y 7 columnas

    function randomFn(x,y) # Calcula la aplicación aleatoria de las contracciones a un punto (x,y)
        r = rand()
        treshold = 0.
        for n in 1:n_fns
            if r <= (treshold += coeffs[n,7]) # coeffs[n,7] es la probabilidad asociada a una contracción
                x_tmp = x
                x = coeffs[n,1]*x + coeffs[n,2]*y + coeffs[n,5] # Aplicación de la contracción a x
                y = coeffs[n,3]*x_tmp + coeffs[n,4]*y + coeffs[n,6] # Aplicación de la contracción a y
                return x,y
            end
        end
        x,y
    end
    
    img = Gray.(ones(height, width)) # Imagen blanca
    
    x_min = -1. # Área para dibujar: [-1,1]x[-1,1]
    x_max = 1.
    y_min = -1.
    y_max = 1.

    x = 0.
    y = 0.
    
    for k in 1:200 # Hay que calcular nuevos límites para el área, para que quepa o se vea el atractor
        x,y = randomFn(x,y)
        if x < x_min x_min = x end
        if x > x_max x_max = x end
        if y < y_min y_min = y end
        if y > y_max y_max = y end
    end
    
    i = 0
    j = 0
        
    for k in 1:n_points
        i = ceil(Int64, width*(x - x_min)/(x_max - x_min)) # Conversión de coordenada "x" a "i" de imagen
        j = height - floor(Int64, height*(y - y_min)/(y_max - y_min))
        if 1 <= i <= width && 1 <= j <= height # Si no está dentro de estos índices, no está en la imagen
            img[j,i] = 0.
        end
        x,y = randomFn(x,y)
    end
    
    img
end

In [ ]:
ifs_koch = [
0.3333   0.0000   0.0000   0.3333   0.0000  0.0000 0.4;
0.1667  -0.2887   0.2887   0.1667   0.3333  0.0000 0.4;
0.1667   0.2887  -0.2887   0.1667   0.5000  0.2887 0.1;
0.3333   0.0000   0.0000   0.3333   0.6667  0.0000 0.1]

ifs_sierpinski = [
0.5 0 0 0.5  0.00  0.000  0.3333;
0.5 0 0 0.5  0.50  0.000  0.3333;
0.5 0 0 0.5  0.25  0.433  0.3334]

ifs_dragon = [
 0.5  0.5  -0.5  0.5   0  0   0.5;
-0.5  0.5  -0.5 -0.5   1  0   0.5]

ifs_fern = [
 0.00   0.00   0.00  0.16  0.0  0.00   0.01;
 0.20  -0.26   0.23  0.22  0.0  1.60   0.07;
-0.15   0.28   0.26  0.24  0.0  0.44   0.07;    
 0.85   0.04  -0.04  0.85  0.0  1.60   0.85]

ifs_coral = [
-0.16666667 -0.1666667  0.16666667 -0.1666667  0.0000000  0.000000  0.163;
 0.83333333  0.2500000 -0.25000000  0.8333333 -0.1666667 -0.166667  0.600;
 0.33333333 -0.0833333  0.08333333  0.3333333  0.0833333  0.666667  0.237]

ifs_square = [
 0.5  0.0  0.0  0.5  0.0  0.0  0.15;
 0.5  0.0  0.0  0.5  0.5  0.0  0.15;
 0.5  0.0  0.0  0.5  0.0  0.5  0.1;
 0.5  0.0  0.0  0.5  0.5  0.5  0.6]

Cada $n$-ésimo renglón $a_n,b_n,c_n,d_n,t_n,s_n,p_n$ de las matrices anteriores indica la contracción

$f_n(x,y)=(a_n x + b_n y + t_n, c_n x + d_n y + s_n)$, con una probabilidad $p_n$.

In [ ]:
createIFSAttractor(ifs_koch, 2000, 2000, 1000000)

### Ejemplo: El conjunto de Mandelbrot

El conjunto de Mandelbrot es el conjunto
$$\mathcal{M}=\{c \in \mathbb{C}| q_c^n(0) \nrightarrow \infty\}$$
donde $q_c(z) = z^2+c$.

En el algoritmo, dibuja con el gris $n/N_{max}\in [0,1]$ el pixel correspondiente al $c$, donde $n$ es primer entero tal que $||q_c^n(0)||^2>4$. $N_{max}$ es un número máximo de iteraciones dado.

In [ ]:
# Tamaño de la imagen
width = 500
height = 500

# Imagen en blanco
img_mandelbrot = Gray.(ones(height,width))

# Región en el plano [x_min,x_max]x[y_min,y_max]
x_min = -2.2
x_max = 0.8
y_min = -1.5
y_max = 1.5

# Variables para c y z
x = 0.
y = 0.
z = 0.+im
c = 0.+im

# Número máximo de iteraciones
max_iters = 80

# Contador de iteraciones
n = 0

for j in 1:height # Recorriendo el eje Y, la altura de la imagen
    y = y_min + (j-1)*(y_max-y_min)/height
    
    for i in 1:width # Recorriendo el eje X, el ancho de la imagen
        x = x_min + (i-1)*(x_max-x_min)/width
        
        c = x + y*im # Estamos en el plano parámetrico, cada pixel es un c
        z = c # Esto equivale a la primera iteración del 0: z = (0)^2 + c
        
        for n in 1:max_iters # Checando que las iteraciones de 0 estén acotadas
            if abs2(z) > 4 # si |z| > 2, entonces q^n(z) -> Inf
                break
            end
            
            z = z^2 + c
        end
        
        img_mandelbrot[j,i] = n/max_iters # Un número entre 0 y 1, es gris...     
    end
end

img_mandelbrot

### Teorema
Si $z_0\in J(f)$ entonces $\overline{\bigcup f^{-n}(z_0)} = J(f)$. 

In [ ]:
# Dibuja el conjunto de Julia de q_c(z) = z^2 + c, usando el teorema de arriba
function createJulia(z0::Complex, c::Complex, x_min::Real, x_max::Real, y_min::Real, y_max::Real,
    width::Integer=500, height::Integer=500, n_points::Integer=100000)
    
    img = Gray.(zeros(height,width))
    
    z = z0
    
    for n in 1:n_points
        i = ceil(Int64, width*(z.re - x_min)/(x_max - x_min)) # Conversión de coordenada "x" a "i" de imagen
        j = height - floor(Int64, height*(z.im - y_min)/(y_max - y_min))
        
        #if 1 <= i <= width && 1 <= j <= height # Si no está dentro de estos índices, no está en la imagen
        img[j,i] = 1.
        #end
       
        z = rand() < 0.5 ? sqrt(z - c) : -sqrt(z - c)
    end

    img
end

In [ ]:
createJulia(0.9im, -1.+0im, -2., 2., -2., 2.)
# 0.5+0im, 0.25+0im # -1.+0im, 0.5im # 0.5im, -1.+0im

## Imágenes a color

Una manera de especificar colores, es mediante ternas de números llamadas *RGB* (*Red-Green-Blue*).

Por ejemplo (usando números en $[0,1]$)

- $(0,0,0)$ es negro.
- $(1,0,0)$ es rojo brillante.
- $(0.25,0,0)$ es rojo obscuro.
- $(0,0,1)$ es azul brillante.
- $(1,1,0)$ es amarillo.
- $(0.5,0.5,0.5)$ es gris.
- $(1,1,1)$ es blanco.

In [ ]:
color_rgb = RGB(0,0.6,0)

In [ ]:
# Crea una imagen en colores (aleatoria) de 200x100 pixeles
img_rand_rgb = rand(RGB{Float32},100,200)

### Ejemplo: El conjunto de Julia lleno

Dada una función $f:\mathbb{C} \rightarrow \mathbb{C}$, el conjunto de Julia lleno es
$$\mathcal{K}(f)=\{z \in \mathbb{C}| f^n(z) \nrightarrow \infty\}$$

Para este algoritmo, en lugar de un gris $\in [0,1]$, se asignará al pixel un color de un arreglo de colores precalculado, de tamaño en máximo de iteraciones.

Cada color indicará la iteración en la que se "escapa" la iterada...

In [ ]:
# Dada una lista de colores base, crea una lista de N colores usando interpolación
function createColorArray(N::Integer, base_colors::Array)
    n_base_colors = length(base_colors)
    M = round(Int64, N/(n_base_colors-1))+1
    clrs_array = [base_colors[1]]
    for n in 2:n_base_colors
        for k in 1:M
            push!(clrs_array, (1.-k/M)*base_colors[n-1] + (k/M)*base_colors[n])
        end
    end  
    clrs_array
end

In [ ]:
c = -0.70176-0.3842*im
#c = -1.
#c = im
#c = 0.285+0.01*im
#c=0.25
#c = -0.7269+0.1889*im
#c=e/pi

f(z::Complex) = z^2 + c
#f(z::Complex) = z^2 + c/(z^2)
#f(z::Complex) = z - (z^3 - 1.)/(3z^2)
#f(z::Complex) = c*sin(z)
#f(z::Complex) = c*exp(z)

# Tamaño de la imagen
width = 500
height = 500

# Imagen en negro
img_filled_julia = zeros(RGB{Float32},height,width)

# Región en el plano [x_min,x_max]x[y_min,y_max]
x_min = -2.
x_max = 2.
y_min = -2.
y_max = 2.

# Variables z
x = 0.
y = 0.
z = 0.+im

# Número máximo de iteraciones
max_iters = 50

# Arreglo de colores
colors_array = createColorArray(max_iters, [RGB(0,0,0.2), RGB(1,1,1), RGB(1,0,0)])
#colors_array = createColorArray(max_iters, [RGB(1,1,1), RGB(0,0,0), RGB(0,1,1), RGB(1,0,0)])
#colors_array = createColorArray(max_iters, [RGB(0,0,0), RGB(0,0,0.5), RGB(0,1,1), RGB(0,1,0), RGB(1,1,0), RGB(1,0,0), RGB(1,1,1)])

# Contador de iteraciones
n = 0

for j in 1:height # Recorriendo el eje Y, la altura de la imagen
    y = y_min + (j-1)*(y_max-y_min)/height
    
    for i in 1:width # Recorriendo el eje X, el ancho de la imagen
        x = x_min + (i-1)*(x_max-x_min)/width
        
        z = x + y*im # Estamos en el plano dinámico, cada pixel es un z
        
        for n in 1:max_iters # Checando que las iteraciones de z estén acotadas
            if abs2(z) > 4 # si |z| > 2, entonces q^n(z) -> Inf
                break
            end
            
            z = f(z)
        end
        
        img_filled_julia[j,i] = colors_array[n]
    end
    
end

img_filled_julia

Para guardar imágenes creadas o modificadas:

In [ ]:
using FileIO
save("filled_julia.png", img_filled_julia)

## Abrir imágenes de un archivo

In [ ]:
#using FileIO
img_cat = load("cat_colors.png")

### Ejemplo: El mapeo del gato de Arnol'd

El mapeo del gato de Arnol'd es un mapeo en el toro plano bidimensional $\mathbb{T}^2=\mathbb{R}^2/\mathbb{Z}^2$ dado por
$$ \left(\begin{array}{c} x\\ y
\end{array}\right)
\mapsto
\left(\begin{array}{cc}
2 & 1\\
1 & 1
\end{array}\right)
\left(\begin{array}{c} x\\ y
\end{array}\right)\mod\,1$$

In [ ]:
# Iteraciones
N=1

img_size = Int(sqrt(length(img_cat))) # La imagen debe de ser cuadrada
img_cat0 = copy(img_cat) # Copias de la imagen, para no modificar la original
img_cat1 = copy(img_cat)

for n in 1:N
    for j in 1:img_size
        for i in 1:img_size
            ci = (2i + j) % img_size + 1 # Mapeo del gato de Arnol'd
            cj =  (i + j) % img_size + 1
    
            img_cat1[j,i] = img_cat0[cj,ci]
        end
    end
    img_cat0 = copy(img_cat1)
end

img_cat1